In [19]:
import pickle
import os
from tqdm import tqdm 
from typing import Dict
from collections import defaultdict
import numpy as np
from pymongo import MongoClient

In [20]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [21]:
entities = []
id2ent = {}
with open("wd5m_aliases_entities_v3.txt", 'r') as f:
    for line in f:
        entities.append(line.split('\t')[1][:-1])
        id_, ent = line.split('\t')
        id2ent[id_] = ent[:-1]

In [22]:
id2ent['Q13406463'], id2ent['Q34740']

('Wikimedia list article', 'genus')

In [23]:
fname = 'scores/baseline_model.pickle'
# fname = 'scores_500_base_trie.pickle'
scores_data = pickle.load(open(fname, 'rb'))

In [24]:
entity_strings_set = set(entities)

In [10]:
def create_filter_dict(collection_name) -> Dict[str, int]:
    
    client = MongoClient('localhost', 27017)
    collection = client['KGLM'][collection_name]
    
    filter_dict = defaultdict(list)
#     tot = client['KGLM'].command("collstats", collection_name)['count']
#     for doc in tqdm(collection.find({}), total=tot):
    for doc in collection.find({}):
        inp = id2ent[doc['head']]
        filter_dict[inp].append(doc['tail'])
    return filter_dict

def getAllFilteringEntities(inp, filter_dicts):
    entities = []
    splits = ['train', 'test', 'valid']
    for s in splits:
        entities.extend(filter_dicts[s][inp])
    return list(set(entities))

filter_dicts = {}
splits = ['train', 'valid', 'test']
for split in splits:
    filter_dicts[split] = create_filter_dict(split)

Exception ignored in: <function tqdm.__del__ at 0x7f570032a790>
Traceback (most recent call last):
  File "/data/home/admin/t5-experiments-from-yura/venv/lib/python3.8/site-packages/tqdm/std.py", line 1162, in __del__
    self.close()
  File "/data/home/admin/t5-experiments-from-yura/venv/lib/python3.8/site-packages/tqdm/notebook.py", line 288, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


In [25]:
predictions_scores_dicts = []
for string_arr, score_arr in tqdm(zip(scores_data['prediction_strings'], scores_data['scores'])):
    ps_pairs = [(p,s) for p,s in zip(string_arr, score_arr)]
    ps_pairs = list(set(ps_pairs)) # while sampling, duplicates are created
    # remove predictions that are not entities
    ps_dict_only_entities = defaultdict(list)
    for ps in ps_pairs:
        if ps[0] in entity_strings_set:
            ps_dict_only_entities[ps[0]] = ps[1]
    predictions_scores_dicts.append(ps_dict_only_entities)

5133it [00:00, 6603.81it/s]


In [26]:
predictions_filtered = []
head_num_filter = 0
tail_num_filter = 0

for i in tqdm(range(len(predictions_scores_dicts))):
    ps_dict = predictions_scores_dicts[i].copy()
    target = scores_data['target_strings'][i]
    inputs = scores_data['input_strings'][i]
    prediction_strings = ps_dict.keys()
    
    if target in prediction_strings:
        original_score = ps_dict[target]
    # get filtering entities
    filtering_entities = getAllFilteringEntities(inputs, filter_dicts)

    for ent in filtering_entities:
        if ent in ps_dict:
            ps_dict[ent] = -float("inf")
    if target in prediction_strings:
        ps_dict[target] = original_score
    # softmax for scores
    names_arr = []
    scores_arr = []
    for k, v in ps_dict.items():
        names_arr.append(k)
        scores_arr.append(v)
    scores_arr = np.array(scores_arr)
#     scores_arr = softmax(scores_arr)
    for name, score in zip(names_arr, scores_arr):
        ps_dict[name] = score
    predictions_filtered.append(ps_dict)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5133/5133 [00:00<00:00, 10286.67it/s]


In [27]:
count = {}
reciprocal_ranks = 0.0
k_list = [1, 3, 10]

for k in k_list:
    count[k] = 0
    
num_small_arrs = 0
total_count = 0

for i in tqdm(range(len(predictions_filtered))):
    target = scores_data['target_strings'][i]
    ps_dict = predictions_filtered[i]
    ps_sorted = sorted(ps_dict.items(), key=lambda item: -item[1])
    inputs = scores_data['input_strings'][i]

    if len(ps_dict) == 0:
        preds = []
    else:
        preds = [x[0] for x in ps_sorted]
    if len(filtering_entities) > 1 and target in preds[:1]:
        print(i)
    if target in preds:
        rank = preds.index(target) + 1
        reciprocal_ranks += 1./rank
    for k in k_list:
        if target in preds[:k]:
            count[k] += 1
    if len(preds) < 10 and target not in preds:
        num_small_arrs += 1
        
total_count = len(predictions_filtered)
for k in k_list:
    hits_at_k = count[k]/total_count
    print('hits@{}'.format(k), hits_at_k)
print('mrr', reciprocal_ranks/total_count)
print(num_small_arrs/total_count, 'were <10 length preds array without answer')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5133/5133 [00:00<00:00, 62631.05it/s]

hits@1 0.11805961426066627
hits@3 0.19286966686148452
hits@10 0.2577440093512566
mrr 0.1655579625790256
0.13208649912331968 were <10 length preds array without answer


In [18]:
count = {}
reciprocal_ranks = 0.0
k_list = [1, 3, 10]

for k in k_list:
    count[k] = 0
    
num_small_arrs = 0
total_count = 0

for i in tqdm(range(len(predictions_filtered))):
    target = scores_data['target_strings'][i]
    ps_dict = predictions_filtered[i]
    ps_sorted = sorted(ps_dict.items(), key=lambda item: -item[1])
    inputs = scores_data['input_strings'][i]

    if len(ps_dict) == 0:
        preds = []
    else:
        preds = [x[0] for x in ps_sorted]
    if len(filtering_entities) > 1 and target in preds[:1]:
        print(i)
    if target in preds:
        rank = preds.index(target) + 1
        reciprocal_ranks += 1./rank
    for k in k_list:
        if target in preds[:k]:
            count[k] += 1
    if len(preds) < 10 and target not in preds:
        num_small_arrs += 1
        
total_count = len(predictions_filtered)
for k in k_list:
    hits_at_k = count[k]/total_count
    print('hits@{}'.format(k), hits_at_k)
print('mrr', reciprocal_ranks/total_count)
print(num_small_arrs/total_count, 'were <10 length preds array without answer')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5133/5133 [00:00<00:00, 398308.34it/s]

hits@1 0.0
hits@3 0.0
hits@10 0.0
mrr 0.0
1.0 were <10 length preds array without answer
